In [1]:
import os

os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [3]:
from deepClassifier.constants import *
from deepClassifier.utils import create_directories, read_yaml

In [4]:
from importlib.resources import path


class ConfigurationManager:
    def __init__(
        self, config_file_path=CONFIG_FILE_PATH, param_file_path=PARAMS_FILE_PATH
    ) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir), model_ckpt_dir])
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )
        return prepare_callback_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile

import tensorflow as tf

2022-09-22 04:27:05.240799: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-22 04:27:05.344958: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-22 04:27:05.669217: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pk/miniconda3/lib/:/home/pk/miniconda3/lib/:/home/pk/miniconda3/envs/deep/lib/
2022-09-22 04:27:05.669290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic li

In [6]:
import time


class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_log_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath , 
            save_best_only=True
        )

    def get_tb_ckpt_callback(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]

In [7]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    prepare_callbacks.get_tb_ckpt_callback()
except Exception as e:
    raise e

2022-09-22 04:27:06.115 | INFO     | deepClassifier.utils.common:read_yaml:30 - yaml file: configs/config.yaml loaded successfully
2022-09-22 04:27:06.116 | INFO     | deepClassifier.utils.common:read_yaml:30 - yaml file: params.yaml loaded successfully
2022-09-22 04:27:06.117 | INFO     | deepClassifier.utils.common:create_directories:49 - created directory at: artifacts
2022-09-22 04:27:06.117 | INFO     | deepClassifier.utils.common:create_directories:49 - created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
2022-09-22 04:27:06.118 | INFO     | deepClassifier.utils.common:create_directories:49 - created directory at: artifacts/prepare_callbacks/checkpoint_dir
